## Test基本功能

In [226]:
import pennylane as qml
from pennylane import numpy as np

In [51]:
#測試control unitary的功能
U = np.array([[0, 0, 0, 1],
              [1, 0, 0, 0],
              [0, 1, 0, 0],
              [0, 0, 1, 0]])

qml.ControlledQubitUnitary(U, control_wires = [0, 1], wires = [2, 3])


ControlledQubitUnitary(tensor([[0, 0, 0, 1],
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0]], requires_grad=True), wires=[0, 1, 2, 3])

In [97]:
#創建quantum circuit
dev = qml.device('default.qubit', wires = 4, shots = 8)



@qml.qnode(dev)
def circuit():
    qml.Hadamard(1)
    qml.ControlledQubitUnitary(U, control_wires = [0,1], wires = [2,3], control_values='01')
    return [qml.sample(qml.PauliZ(i)) for i in range(4)]


In [98]:
drawer = qml.draw(circuit)
print(drawer())

 0: ─────╭C───┤ Sample[Z] 
 1: ──H──├C───┤ Sample[Z] 
 2: ─────├U0──┤ Sample[Z] 
 3: ─────╰U0──┤ Sample[Z] 
U0 =
[[0 0 0 1]
 [1 0 0 0]
 [0 1 0 0]
 [0 0 1 0]]



In [100]:
A = circuit()

In [101]:
A

array([[ 1,  1,  1,  1,  1,  1,  1,  1],
       [ 1, -1,  1, -1,  1, -1,  1, -1],
       [ 1,  1,  1,  1,  1,  1,  1,  1],
       [ 1, -1,  1, -1,  1, -1,  1, -1]], dtype=int64)

In [105]:
#將eigenvalue的形式轉成basis的形式
B = np.transpose(np.where(A==1,0,1))
B

tensor([[0, 0, 0, 0],
        [0, 1, 0, 1],
        [0, 0, 0, 0],
        [0, 1, 0, 1],
        [0, 0, 0, 0],
        [0, 1, 0, 1],
        [0, 0, 0, 0],
        [0, 1, 0, 1]], requires_grad=True)

## 筆記紀錄
在pennylane使用pauli Z measurement，得到的1對應到qiskit measurement中的0;得到的-1對應到qiskit measurement中的1(eigenvalue、eigenvector的關係)
然後結果的順序相對於qiskit而言是比較直觀的!在qiskit上到下對應到右到左，而在pennylane上到下對應到左到右


In [34]:
dev = qml.device("default.qubit", wires=2)

@qml.qnode(dev)
def circuit():
    qml.Hadamard(wires=1)
    return qml.sample([qml.PauliZ(0),[0,1]])

In [35]:
circuit()

AttributeError: 'list' object has no attribute 'name'

In [228]:
#PENNYLANE TRAINGNIG CONTROL U3-GATE 測試

dev = qml.device("default.qubit.autograd", wires=3, shots = 10)

#U3的代表矩陣型式
params = np.random.rand(6)*np.pi

#U3_1 = np.array([[np.cos(params[0]/2),-1*np.exp(1j*params[2])*np.sin(params[0]/2)],
               #[np.exp(1j*params[1])*np.sin(params[0]/2),np.exp(1j*(params[1]+params[2]))*np.cos(params[0]/2)]])

#U3_2 = np.array([[np.cos(params[3]/2),-1*np.exp(1j*params[5])*np.sin(params[3]/2)],
               #[np.exp(1j*params[4])*np.sin(params[3]/2),np.exp(1j*(params[4]+params[5]))*np.cos(params[3]/2)]])
@qml.qnode(dev)
def circuit(params):
    U3_1 = np.array([[np.cos(params[0]/2),-1*np.exp(1j*params[2])*np.sin(params[0]/2)],
                     [np.exp(1j*params[1])*np.sin(params[0]/2),np.exp(1j*(params[1]+params[2]))*np.cos(params[0]/2)]])

    U3_2 = np.array([[np.cos(params[3]/2),-1*np.exp(1j*params[5])*np.sin(params[3]/2)],
                     [np.exp(1j*params[4])*np.sin(params[3]/2),np.exp(1j*(params[4]+params[5]))*np.cos(params[3]/2)]])
    qml.Hadamard(0) 
    qml.ControlledQubitUnitary(U3_1, control_wires=0, wires=1)
    qml.ControlledQubitUnitary(U3_2, control_wires=0, wires=2)
    #return [qml.sample(qml.PauliZ(i+1)) for i in range(2)]
    return  [qml.expval(qml.PauliZ(i)) for i in range(2)]
def cost(params):
    A = circuit(params)
    print(A)
    return np.sum(A)
    #B = np.where(A==1,0,1)
    #print(B)
    #return -1 * np.sum(B)

#init_params = np.random.rand(6)*np.pi
#print(init_prams)
print(cost(params))

[0.2 1. ]
1.2


In [229]:
#試著更新上面我們自己定義的lost function
opt = qml.GradientDescentOptimizer(stepsize=0.2)
steps = 100
for i in range(steps):
    #更新U3 GATE的參數
    params = opt.step(cost, params)

    if (i + 1) % 5 == 0:
        print("Cost after step {:5d}: {: .7f}".format(i + 1, cost(params)))

print("Optimized rotation angles: {}".format(params))

##感覺問題出在自訂義U3那邊

TypeError: loop of ufunc does not support argument 10 of type ArrayBox which has no callable conjugate method

In [180]:
dev1 = qml.device("default.qubit", wires=2, shots = 10)

In [223]:
params = np.random.rand(6)*np.pi

@qml.qnode(dev1)
def circuit(params):
    qml.Hadamard(0)
    qml.RX(params[0], wires=0)
    qml.RY(params[1], wires=0)
    qml.RZ(params[2], wires=0)
    qml.RX(params[3], wires=1)
    qml.RY(params[4], wires=1)
    qml.RZ(params[5], wires=1)
    return  [qml.expval(qml.PauliZ(i)) for i in range(2)]
def cost(x):
    A = circuit(x)
    return np.sum(A)
 
    #B = np.where(A==1,0,1)
    #print(B)
    #return -1 * np.sum(B)

#init_params = np.random.rand(6)*np.pi
#print(init_prams)
print(cost(params))

0.6


In [225]:
#試著更新上面我們自己定義的lost function
opt = qml.GradientDescentOptimizer(stepsize=0.2)
steps = 100
for i in range(steps):
    #更新U3 GATE的參數
    params = opt.step(cost, params)

    if (i + 1) % 5 == 0:
        print("Cost after step {:5d}: {: .7f}".format(i + 1, cost(params)))

print("Optimized rotation angles: {}".format(params))

##感覺問題出在自訂義U3那邊

Cost after step     5: -1.8000000
Cost after step    10: -2.0000000
Cost after step    15: -2.0000000
Cost after step    20: -2.0000000
Cost after step    25: -2.0000000
Cost after step    30: -2.0000000
Cost after step    35: -2.0000000
Cost after step    40: -2.0000000
Cost after step    45: -2.0000000
Cost after step    50: -2.0000000
Cost after step    55: -2.0000000
Cost after step    60: -2.0000000
Cost after step    65: -2.0000000
Cost after step    70: -2.0000000
Cost after step    75: -2.0000000
Cost after step    80: -2.0000000
Cost after step    85: -2.0000000
Cost after step    90: -2.0000000
Cost after step    95: -2.0000000
Cost after step   100: -2.0000000
Optimized rotation angles: [ 0.11299497  1.47750672  1.40707171  3.12761435 -0.04125121  0.92007642]


In [133]:
#create custom gate
import pennylane as qml
from pennylane.operation import Operation
from pennylane import numpy as np

class ControlUnitary(Operation):
    num_params = 1
    num_wires = 1
    par_domain = "R"
    
    @staticmethod
    def decomposition(U,wires):
        return [qml.QubitUnitary(U, wires=wires)]

    
    grad_method = "A"
    grad_recipe = None # This is the default but we write it down explicitly here.

In [134]:
dev5 = qml.device("default.qubit", wires=1, shots = 10)

params = np.random.rand(3)*np.pi
@qml.qnode(dev5)
def circuit(params):
    U3 = np.array([[np.cos(params[0]/2),-1*np.exp(1j*params[2])*np.sin(params[0]/2)],
                     [np.exp(1j*params[1])*np.sin(params[0]/2),np.exp(1j*(params[1]+params[2]))*np.cos(params[0]/2)]])
    qml.Hadamard(0)
    ControlUnitary(U3,wires=0)
    return [qml.expval(qml.PauliZ(i)) for i in range(1)]
def cost(x):
    A = circuit(x)
    print(A)
    return np.sum(A)

In [135]:
drawer = qml.draw(circuit)
print(drawer)

<function circuit at 0x0000025B716E6E50>


In [136]:
circuit(params)

tensor([0.2], requires_grad=True)

In [128]:
print(cost(params))

[0.8]
0.8


In [129]:
opt = qml.GradientDescentOptimizer(stepsize=0.2)
steps = 100
for i in range(steps):
    #更新U3 GATE的參數
    params = opt.step(cost, params)

    if (i + 1) % 5 == 0:
        print("Cost after step {:5d}: {: .7f}".format(i + 1, cost(params)))

print("Optimized rotation angles: {}".format(params))

Autograd ArrayBox with value [0.8]


ValueError: Cannot differentiate with respect to parameter(s) {0}

In [130]:
import pennylane as qml
from pennylane.operation import Operation
from pennylane import numpy as np

class RXX(Operation):
    num_params = 1
    num_wires = 2
    par_domain = "R"

    @staticmethod
    def decomposition(theta, wires):
        return [qml.PauliRot(theta, 'XX', wires=wires)]

In [131]:
dev = qml.device('default.qubit', wires=2, shots=100)

@qml.qnode(dev)
def circuit(theta):
    RXX(theta, wires=[0, 1])
    qml.Hadamard(1)
    return qml.expval(qml.PauliZ(0) @ qml.PauliY(1))

In [132]:
circuit(0.3)

tensor(0.16, requires_grad=True)